In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic
from pathlib import Path
from anthropic.types import Message
from IPython.display import Markdown

load_dotenv()

client = Anthropic(
    default_headers={
        "anthropic-beta": "code-execution-2025-05-22, files-api-2025-04-14"
    }
)
model = "claude-sonnet-4-20250514"

In [2]:
# Helper functions

def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=2000,
):
    params = {
        "model": model,
        "max_tokens": 10000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )


def upload(file_path):
    path = Path(file_path)
    extension = path.suffix.lower()

    mime_type_map = {
        ".pdf": "application/pdf",
        ".txt": "text/plain",
        ".md": "text/plain",
        ".py": "text/plain",
        ".js": "text/plain",
        ".html": "text/plain",
        ".css": "text/plain",
        ".csv": "text/csv",
        ".json": "application/json",
        ".xml": "application/xml",
        ".xlsx": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        ".xls": "application/vnd.ms-excel",
        ".jpeg": "image/jpeg",
        ".jpg": "image/jpeg",
        ".png": "image/png",
        ".gif": "image/gif",
        ".webp": "image/webp",
    }

    mime_type = mime_type_map.get(extension)

    if not mime_type:
        raise ValueError(f"Unknown mimetype for extension: {extension}")
    filename = path.name

    with open(file_path, "rb") as file:
        return client.beta.files.upload(file=(filename, file, mime_type))


def list_files():
    return client.beta.files.list()


def delete_file(id):
    return client.beta.files.delete(id)


def download_file(id, filename=None):
    file_content = client.beta.files.download(id)

    if not filename:
        file_metadata = get_metadata(id)
        file_content.write_to_file(file_metadata.filename)
    else:
        file_content.write_to_file(filename)


def get_metadata(id):
    return client.beta.files.retrieve_metadata(id)

In [9]:
file_meta = upload("streaming.csv")
file_meta.id

'file_011CUWWan9jJz7hucTTJ7X9c'

In [24]:
messages = []

add_user_message(
    messages,
    [
        {
            "type": "text",
            "text": """
Run a detailed analysis to determine major drivers of churn.
Your final output should include at least one detailed plot summarizing your findings.

Critical note: Every time you execute code, you're starting with a completely clean slate. 
No variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.
            """,
        },
        {"type": "container_upload", "file_id": file_meta.id},
    ],
)

chat(
    messages,
    tools=[{"type": "code_execution_20250522", "name": "code_execution"}]
)

Message(id='msg_01FJGACmPGrXFknuJmwDmot7', content=[TextBlock(citations=None, text="I'll analyze the churn data to identify major drivers of customer churn. Let me start by exploring the dataset and then conduct a comprehensive analysis.", type='text'), ServerToolUseBlock(id='srvtoolu_0146WD5puCxZadsmk4oeirbJ', input={'code': 'import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import classification_report, confusion_matrix\nimport warnings\nwarnings.filterwarnings(\'ignore\')\n\n# Set up plotting style\nplt.style.use(\'default\')\nsns.set_palette("husl")\n\n# Load the data\nimport os\ndata = pd.read_csv(os.path.join(os.environ[\'INPUT_DIR\'], \'streaming.csv\'))\n\n# Initial exploration\nprint("Dataset Shape:", data.shape)\nprint("\\nColumn Names:")\nprint(data.column

In [41]:
attempt = messages
dd = attempt
print(len(dd))
ini = dd[-1]

1


In [48]:
ini
ant = [i for i in ini.items()]

In [60]:
print(ant[1])
print(type(ant[1][1]))
print(ant[1][1][0])
print(ant[1][1][1])
print(type(ant[1][1][0]))
print(ant[1][1][1]["file_id"])

('content', [{'type': 'text', 'text': "\nRun a detailed analysis to determine major drivers of churn.\nYour final output should include at least one detailed plot summarizing your findings.\n\nCritical note: Every time you execute code, you're starting with a completely clean slate. \nNo variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.\n            "}, {'type': 'container_upload', 'file_id': 'file_011CUWWan9jJz7hucTTJ7X9c'}])
<class 'list'>
{'type': 'text', 'text': "\nRun a detailed analysis to determine major drivers of churn.\nYour final output should include at least one detailed plot summarizing your findings.\n\nCritical note: Every time you execute code, you're starting with a completely clean slate. \nNo variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.\n            "}
{'type': 'container_upload', 'file_id': 'file_011CUWWan9jJz7hucTTJ7X9c'}
<clas

In [ ]:
import json
with open("messages.json", "w") as j:
   j.write(json.dumps(messages, indent=3))
json.dumps(messages, indent=2)





'[\n  {\n    "role": "user",\n    "content": [\n      {\n        "type": "text",\n        "text": "\\nRun a detailed analysis to determine major drivers of churn.\\nYour final output should include at least one detailed plot summarizing your findings.\\n\\nCritical note: Every time you execute code, you\'re starting with a completely clean slate. \\nNo variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.\\n            "\n      },\n      {\n        "type": "container_upload",\n        "file_id": "file_011CUKYPA9Qhwnu8YKAPQ6sH"\n      }\n    ]\n  }\n]'

In [ ]:
with open("messages", "w") as m:
   m.write([m for m in messages])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3667, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1869454/3799255988.py", line 2, in <module>
    m.write([m for m in messages])
    ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
TypeError: write() argument must be str, not list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2176, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
    

In [45]:
with open("messages", "w") as f:
   f.write(messages[0].items().mapping)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3667, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1869454/3176473943.py", line 2, in <module>
    f.write(messages[0].items().mapping)
    ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: write() argument must be str, not mappingproxy

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2176, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/ultratb.py", line 1182, in structured_traceback
    return FormattedTB.struct

In [70]:
attempt = [messages[i].items() for i in range(len(messages))]
len(attempt)

1

In [71]:
ditems = attempt[0]

In [82]:
ditems.mapping.items()

dict_items([('role', 'user'), ('content', [{'type': 'text', 'text': "\nRun a detailed analysis to determine major drivers of churn.\nYour final output should include at least one detailed plot summarizing your findings.\n\nCritical note: Every time you execute code, you're starting with a completely clean slate. \nNo variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.\n            "}, {'type': 'container_upload', 'file_id': 'file_011CUKYPA9Qhwnu8YKAPQ6sH'}])])